This notebook just contains the final results, the background works which help to conclude that this structures can have these accuracies are in other Notebooks.

for PART1 we used dense-net like structure which learned on image-net (as transfer learning) and trained 3 different individual to train a ensemble model.
in PART2 however, we just used mixcut augmentation, criterion, and some other regularization methods.
as we examined in other Notebooks, with method we used here in part1, we got lower accuracy in part2.
also with method we used here in part2, we got lower accuracy if use it in part1 too.

##PART1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        out = torch.cat([x, out], 1)
        return out

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers):
        super(DenseBlock, self).__init__()
        layers = []
        for i in range(n_layers):
            layers.append(DenseLayer(in_channels + i * growth_rate, growth_rate))
        self.layer = nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        out = self.pool(out)
        return out

class CustomDenseNet(nn.Module):
    def __init__(self, num_classes=43, growth_rate=32, block_layers=[6, 12, 24, 16]):
        super(CustomDenseNet, self).__init__()
        self.growth_rate = growth_rate
        num_blocks = len(block_layers)

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Dense blocks and transition layers
        in_channels = 64
        self.dense_blocks = nn.ModuleList()
        self.trans_layers = nn.ModuleList()
        for i in range(num_blocks):
            self.dense_blocks.append(DenseBlock(in_channels, growth_rate, block_layers[i]))
            in_channels += growth_rate * block_layers[i]
            if i != num_blocks - 1:  # No transition layer after the last dense block
                out_channels = in_channels // 2
                self.trans_layers.append(TransitionLayer(in_channels, out_channels))
                in_channels = out_channels

        self.bn2 = nn.BatchNorm2d(in_channels)
        self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))
        for i in range(len(self.dense_blocks)):
            x = self.dense_blocks[i](x)
            if i != len(self.dense_blocks) - 1:
                x = self.trans_layers[i](x)
        x = self.relu(self.bn2(x))
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

class EarlyStopping:
    def __init__(self, patience=15, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        self.best_model_wts = model.state_dict()
        self.val_loss_min = val_loss

    def get_best_model_wts(self):
        return self.best_model_wts

def train_model_customCNN(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=25, patience=15):
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    early_stopping = EarlyStopping(patience=patience)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = val_correct / val_total
        val_losses.append(val_epoch_loss)
        val_accuracies.append(val_epoch_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, '
              f'Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}')

        early_stopping(val_epoch_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            model.load_state_dict(early_stopping.get_best_model_wts())
            break

        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_model_wts = model.state_dict()

        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(val_epoch_loss)  # Pass validation loss to scheduler
        else:
            scheduler.step()

    model.load_state_dict(best_model_wts)
    return model, train_losses, val_losses, train_accuracies, val_accuracies

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    misclassifications = defaultdict(list)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Collect misclassified examples
            for label, prediction in zip(labels, predicted):
                if label != prediction:
                    misclassifications[label.item()].append(prediction.item())

    acc = correct / total
    print(f'Test Accuracy: {acc:.4f}')

    # Print misclassification details
    for label, predictions in misclassifications.items():
        print(f'{len(predictions)} data from class number {label} wrongly labeled as {predictions}')


In [ ]:
# Define training function
def train_model_imagenet(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=20):
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    early_stopping = EarlyStopping(patience=15)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = val_correct / val_total
        val_losses.append(val_epoch_loss)
        val_accuracies.append(val_epoch_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, '
              f'Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}')

        scheduler.step(val_epoch_acc)

        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            torch.save(model.state_dict(), 'best_model.pth')
            torch.save(optimizer.state_dict(), 'best_opt.pth')

    print(f'Best Validation Accuracy: {best_val_acc:.4f}')
    return model, train_losses, val_losses, train_accuracies, val_accuracies


####LEARN ON IMAGENET

In [ ]:
epochs = 40
model_copy = CustomDenseNet(1000)  # Replace with model class

# Load the state_dict into the new model instance
model_copy.load_state_dict(torch.load('/kaggle/working/best_model.pth'))

optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
# Load the optimizer state_dict
#checkpoint = torch.load('/kaggle/working/best_model.pth')
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model_copy.to(device)
new_model, train_losses, val_losses, train_accuracies, val_accuracies = train_model_imagenet(model_copy, nn.CrossEntropyLoss(), optimizer, scheduler, test_loader_imgnt, val_loader_imgnt, num_epochs=epochs)


Epoch [1/40] Train Loss: 4.3267, Train Acc: 0.2360, Val Loss: 4.1243, Val Acc: 0.2679
Epoch [2/40] Train Loss: 4.3267, Train Acc: 0.2360, Val Loss: 4.1243, Val Acc: 0.2679
Epoch [3/40] Train Loss: 4.3267, Train Acc: 0.2360, Val Loss: 4.1243, Val Acc: 0.2679


KeyboardInterrupt: 

In [ ]:
from pathlib import Path
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from typing import List, Tuple
import numpy as np
from collections import Counter
import random

num_classes = 43

class GroceryStoreDataset(Dataset):
    def __init__(self, samples: List[Tuple[str, int]] = None, split: str = None, transform=None) -> None:
        super().__init__()
        self.root = Path("/kaggle/working/GroceryStoreDataset/dataset")
        self.transform = transform

        if samples is not None:
            self.samples = samples
            self.labels = [label for _, label in samples]
            self.paths = [path for path, _ in samples]
        else:
            self.split = split
            self.paths, self.labels = self.read_file()

        self.class_names = self.get_class_names()

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []
        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                path, _, label = line.strip().split(", ")
                paths.append(path)
                labels.append(int(label))
        return paths, labels

    def get_class_names(self) -> List[str]:
        class_names = {}
        with open(self.root / "train.txt") as f:
            for line in f:
                path, _, label = line.strip().split(", ")
                if int(label) not in class_names:
                    class_names[int(label)] = Path(path).parent.name
        return [class_names[i] for i in range(len(class_names))]

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

# Sample dataset loader
def sample_max_50(samples, dynamic_size=50):
    sampled_samples = []
    class_count = Counter([label for _, label in samples])
    class_samples = {k: [] for k in class_count.keys()}

    for img_path, label in samples:
        class_samples[label].append((img_path, label))

    for cls in class_samples.keys():
        sampled_samples.extend(random.sample(class_samples[cls], min(dynamic_size, class_count[cls])))

    return sampled_samples

def filter_samples(samples, target_classes):
    return [(img_path, label) for img_path, label in samples if label in target_classes]

# Classes with high and low number of instances (as an example)
#high_instance_classes = [0, 7, 13, 19, 20, 27, 38, 39, 41]
#low_instance_classes = [10, 15, 24, 28, 30, 34, 40]
high_instance_classes = [0, 19, 20, 27, 7, 41, 13, 38, 39]
low_instance_classes = [24, 42, 11, 33, 14, 16, 34, 36, 15, 29, 10, 31, 35, 40, 28]
l3 = high_instance_classes+low_instance_classes
rest_instances = list(range(0,43))
for i in l3:
  rest_instances.remove(i)


# Paths to dataset splits
path2train = "/kaggle/working/GroceryStoreDataset/dataset/train"
path2val = "/kaggle/working/GroceryStoreDataset/dataset/val"
path2test = "/kaggle/working/GroceryStoreDataset/dataset/test"

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}


# Load the datasets
train_dataset_full = GroceryStoreDataset(split='train', transform=data_transforms['train'])
val_dataset_full = GroceryStoreDataset(split='val', transform=data_transforms['val'])
test_dataset_full = GroceryStoreDataset(split='test', transform=data_transforms['test'])

# Prepare samples
train_samples = list(zip(train_dataset_full.paths, train_dataset_full.labels))
val_samples = list(zip(val_dataset_full.paths, val_dataset_full.labels))

train_samples_50 = sample_max_50(train_samples, 36)
train_samples_high = filter_samples(train_samples, high_instance_classes)
train_samples_low = filter_samples(train_samples, low_instance_classes)

# Create datasets using the filtered samples
train_dataset_50 = GroceryStoreDataset(samples=train_samples_50, transform=data_transforms['train'])
train_dataset_high = GroceryStoreDataset(samples=train_samples_high, transform=data_transforms['train'])
train_dataset_low = GroceryStoreDataset(samples=train_samples_low, transform=data_transforms['train'])

# Create DataLoaders
train_loader_50 = DataLoader(train_dataset_50, batch_size=32, shuffle=True, num_workers=4)
train_loader_high = DataLoader(train_dataset_high, batch_size=32, shuffle=True, num_workers=4)
train_loader_low = DataLoader(train_dataset_low, batch_size=32, shuffle=True, num_workers=4)

# Prepare validation samples
val_samples_50 = sample_max_50(val_samples)
val_samples_high = filter_samples(val_samples, high_instance_classes)
val_samples_low = filter_samples(val_samples, low_instance_classes)

# Create validation datasets using the filtered samples
val_dataset_50 = GroceryStoreDataset(samples=val_samples_50, transform=data_transforms['val'])
val_dataset_high = GroceryStoreDataset(samples=val_samples_high, transform=data_transforms['val'])
val_dataset_low = GroceryStoreDataset(samples=val_samples_low, transform=data_transforms['val'])

# Create validation DataLoaders
val_loader_50 = DataLoader(val_dataset_50, batch_size=32, shuffle=False, num_workers=4)
val_loader_high = DataLoader(val_dataset_high, batch_size=32, shuffle=False, num_workers=4)
val_loader_low = DataLoader(val_dataset_low, batch_size=32, shuffle=False, num_workers=4)

# Now can proceed with training and evaluation as before
train_dataset = GroceryStoreDataset(split='train', transform=data_transforms['train'])
val_dataset = GroceryStoreDataset(split='val', transform=data_transforms['val'])
test_dataset = GroceryStoreDataset(split='test', transform=data_transforms['test'])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)


In [ ]:
l3 = high_instance_classes+low_instance_classes
rest_instances = list(range(0,43))
for i in l3:
  rest_instances.remove(i)

alternative_data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

train_samples_rest = filter_samples(train_samples, rest_instances)
train_dataset_rest = GroceryStoreDataset(samples=train_samples_rest, transform=alternative_data_transforms['train'])
train_loader_rest = DataLoader(train_dataset_rest, batch_size=32, shuffle=True, num_workers=4)
val_samples_rest = filter_samples(val_samples, rest_instances)
val_dataset_rest = GroceryStoreDataset(samples=val_samples_rest, transform=alternative_data_transforms['val'])
val_loader_rest = DataLoader(val_dataset_rest, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
model_copy = CustomDenseNet(1000)  # Replace with model class

# Load the state_dict into the new model instance
model_copy.load_state_dict(torch.load('/kaggle/working/best_model.pth'))




<All keys matched successfully>

In [ ]:
model2 = CustomDenseNet(1000)  # Replace with model class

# Load the state_dict into the new model instance
model2.load_state_dict(torch.load('/kaggle/working/best_model.pth'))

<All keys matched successfully>

In [ ]:
model3 = CustomDenseNet(1000)  # Replace with model class

# Load the state_dict into the new model instance
model3.load_state_dict(torch.load('/kaggle/working/best_model.pth'))

<All keys matched successfully>

In [ ]:
def copynn(model):
    model_copy = CustomDenseNet()  # Create a new instance of model
    model_copy.load_state_dict(model.state_dict())
    return model_copy
class EnsembleModelComplex(nn.Module):
    def __init__(self, model_50, model_high, model_low, num_classes):
        super(EnsembleModelComplex, self).__init__()
        self.model_50 = model_50[0]
        self.model_high = model_high[0]
        self.model_low = model_low[0]

        for param in self.model_50.parameters():
            param.requires_grad = False
        for param in self.model_high.parameters():
            param.requires_grad = False
        for param in self.model_low.parameters():
            param.requires_grad = False

        # Unfreeze the last 15 layers of each model
        for param in list(self.model_50.parameters())[-15:]:
            param.requires_grad = True
        for param in list(self.model_high.parameters())[-15:]:
            param.requires_grad = True
        for param in list(self.model_low.parameters())[-15:]:
            param.requires_grad = True


        # Define a more complex combination network
        self.fc1 = nn.Linear(num_classes * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        out_50 = self.model_50(x)
        out_high = self.model_high(x)
        out_low = self.model_low(x)

        out = torch.cat((out_50, out_high, out_low), dim=1)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.fc3(out)

        return out

def train_ensemble_model_complex(ensemble_model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25, patience=5):
    best_val_acc = 0.0
    best_model_weights = None
    no_improvement = 0

    for epoch in range(num_epochs):
        ensemble_model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = ensemble_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')
        scheduler.step()

        # Validation
        val_acc = evaluate_model_complex(ensemble_model, val_loader)
        print(f'Validation Accuracy: {val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_weights = ensemble_model.state_dict().copy()
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print('Early stopping due to no improvement')
            break

    if best_model_weights:
        ensemble_model.load_state_dict(best_model_weights)

    return ensemble_model

def evaluate_model_complex(ensemble_model, val_loader):
    ensemble_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = ensemble_model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return correct / total

In [ ]:
import copy

model_copy1 = copy.deepcopy(model_rest)
model_copy2 = copy.deepcopy(model_high_dens)
model_copy3 = copy.deepcopy(model_low_dens)

In [ ]:
epochs = 35
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_low_dens = model#CustomDenseNet(num_classes)
model_low_dens = model_low_dens.to(device)
optimizer_low = optim.AdamW(model_low_dens.parameters(), lr=0.001, weight_decay=0.01)
scheduler_low = ReduceLROnPlateau(optimizer_low, mode='max', factor=0.1, patience=5, verbose=True)
#model_high = train_individual_model(model_high, train_loader_high, nn.CrossEntropyLoss(), optimizer_high, scheduler_high)
model_low_dens = train_model_customCNN(model_low_dens, nn.CrossEntropyLoss(), optimizer_low, scheduler_low, train_loader_low, val_loader_low, num_epochs=epochs)


Epoch [1/35] Train Loss: 2.7519, Train Acc: 0.4046, Val Loss: 2.2759, Val Acc: 0.3654
Epoch [2/35] Train Loss: 1.3356, Train Acc: 0.8840, Val Loss: 1.8725, Val Acc: 0.5000
Epoch [3/35] Train Loss: 0.7086, Train Acc: 0.9407, Val Loss: 1.6482, Val Acc: 0.5192
Epoch [4/35] Train Loss: 0.4226, Train Acc: 0.9820, Val Loss: 1.6437, Val Acc: 0.5385
Epoch [5/35] Train Loss: 0.2764, Train Acc: 0.9768, Val Loss: 1.6792, Val Acc: 0.5385
Epoch [6/35] Train Loss: 0.2120, Train Acc: 0.9794, Val Loss: 1.6762, Val Acc: 0.5192
Epoch [7/35] Train Loss: 0.1739, Train Acc: 0.9845, Val Loss: 1.6851, Val Acc: 0.5577
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
Epoch [8/35] Train Loss: 0.1564, Train Acc: 0.9871, Val Loss: 1.6955, Val Acc: 0.5577
Epoch [9/35] Train Loss: 0.1336, Train Acc: 1.0000, Val Loss: 1.7365, Val Acc: 0.5577
Epoch [10/35] Train Loss: 0.1478, Train Acc: 0.9820, Val Loss: 1.7070, Val Acc: 0.5577
Epoch [11/35] Train Loss: 0.1501, Train Acc: 0.9871, Val Loss: 1.7384, Val Ac

In [ ]:
dropout_rate = 0.2
num_ftrs = model_copy.fc.in_features
model2.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(num_ftrs, 43)
            )
epochs = 35
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_rest = model2.to(device)#CustomDenseNet(num_classes, growth_rate=32).to(device)
optimizer_rest = optim.AdamW(model_rest.parameters(), lr=0.001, weight_decay=0.01)
scheduler_rest = ReduceLROnPlateau(optimizer_rest, mode='max', factor=0.1, patience=5, verbose=True)

model_rest = train_model_customCNN(model_rest, nn.CrossEntropyLoss(), optimizer_rest, scheduler_rest, train_loader_rest, val_loader_rest, num_epochs=epochs, patience= 10)

Epoch [1/35] Train Loss: 2.8505, Train Acc: 0.2321, Val Loss: 2.3561, Val Acc: 0.1124
Epoch [2/35] Train Loss: 1.8153, Train Acc: 0.4859, Val Loss: 1.8799, Val Acc: 0.3933
Epoch [3/35] Train Loss: 1.2275, Train Acc: 0.6910, Val Loss: 1.6310, Val Acc: 0.5393
Epoch [4/35] Train Loss: 0.8685, Train Acc: 0.7718, Val Loss: 1.3124, Val Acc: 0.5730
Epoch [5/35] Train Loss: 0.6270, Train Acc: 0.8333, Val Loss: 1.3152, Val Acc: 0.5955
Epoch [6/35] Train Loss: 0.4629, Train Acc: 0.8821, Val Loss: 1.2108, Val Acc: 0.6854
Epoch [7/35] Train Loss: 0.3576, Train Acc: 0.9115, Val Loss: 1.2824, Val Acc: 0.6180
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
Epoch [8/35] Train Loss: 0.2701, Train Acc: 0.9282, Val Loss: 1.2309, Val Acc: 0.6292
Epoch [9/35] Train Loss: 0.2609, Train Acc: 0.9436, Val Loss: 1.1780, Val Acc: 0.6629
Epoch [10/35] Train Loss: 0.2500, Train Acc: 0.9449, Val Loss: 1.1999, Val Acc: 0.6966
Epoch [11/35] Train Loss: 0.2083, Train Acc: 0.9551, Val Loss: 1.2407, Val Ac

In [ ]:
dropout_rate = 0.2
num_ftrs = model_copy.fc.in_features
model3.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(num_ftrs, 43)
            )
#model_high_dens = CustomDenseNet(num_classes)
model_high_dens = model3.to(device)#model_high_dens.to(device)
optimizer_high = optim.AdamW(model_high_dens.parameters(), lr=0.001, weight_decay=0.01)
scheduler_high = ReduceLROnPlateau(optimizer_high, mode='max', factor=0.1, patience=5, verbose=True)
#model_high = train_individual_model(model_high, train_loader_high, nn.CrossEntropyLoss(), optimizer_high, scheduler_high)
model_high_dens = train_model_customCNN(model_high_dens, nn.CrossEntropyLoss(), optimizer_high, scheduler_high, train_loader_high, val_loader_high, num_epochs=epochs)

Epoch [1/35] Train Loss: 1.8865, Train Acc: 0.4130, Val Loss: 1.0590, Val Acc: 0.6065
Epoch [2/35] Train Loss: 0.7999, Train Acc: 0.7391, Val Loss: 0.6596, Val Acc: 0.7548
Epoch [3/35] Train Loss: 0.4710, Train Acc: 0.8471, Val Loss: 0.6752, Val Acc: 0.7742
Epoch [4/35] Train Loss: 0.3643, Train Acc: 0.8838, Val Loss: 0.5360, Val Acc: 0.8129
Epoch [5/35] Train Loss: 0.2404, Train Acc: 0.9198, Val Loss: 0.6884, Val Acc: 0.7548
Epoch [6/35] Train Loss: 0.2076, Train Acc: 0.9307, Val Loss: 0.8718, Val Acc: 0.7484
Epoch [7/35] Train Loss: 0.1492, Train Acc: 0.9524, Val Loss: 0.5193, Val Acc: 0.8194
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
Epoch [8/35] Train Loss: 0.1445, Train Acc: 0.9572, Val Loss: 0.4530, Val Acc: 0.8194
Epoch [9/35] Train Loss: 0.1084, Train Acc: 0.9681, Val Loss: 0.4404, Val Acc: 0.8258
Epoch [10/35] Train Loss: 0.0917, Train Acc: 0.9789, Val Loss: 0.4674, Val Acc: 0.8194
Epoch [11/35] Train Loss: 0.0899, Train Acc: 0.9715, Val Loss: 0.4965, Val Ac

##FINAL RESULT FOR PART1

In [ ]:
# Instantiate the ensemble model
ensemble_model_complex = EnsembleModelComplex(model_rest, model_high_dens, model_low_dens, num_classes).to(device)

# Define loss, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer_ensemble = optim.Adam(ensemble_model_complex.parameters(), lr=0.001)  # Optimize all parameters in the complex model
scheduler_ensemble = optim.lr_scheduler.StepLR(optimizer_ensemble, step_size=5, gamma=0.5)

# Train the ensemble model
ensemble_model_complex = train_ensemble_model_complex(ensemble_model_complex, train_loader, val_loader, criterion, optimizer_ensemble, scheduler_ensemble, patience= 10)


Epoch [1/25] Loss: 2.6273, Accuracy: 0.3398
Validation Accuracy: 0.4324
Epoch [2/25] Loss: 1.5975, Accuracy: 0.5545
Validation Accuracy: 0.4865
Epoch [3/25] Loss: 1.1138, Accuracy: 0.6663
Validation Accuracy: 0.5405
Epoch [4/25] Loss: 0.9337, Accuracy: 0.7318
Validation Accuracy: 0.5473
Epoch [5/25] Loss: 0.8208, Accuracy: 0.7705
Validation Accuracy: 0.5034
Epoch [6/25] Loss: 0.5930, Accuracy: 0.8178
Validation Accuracy: 0.5405
Epoch [7/25] Loss: 0.5146, Accuracy: 0.8515
Validation Accuracy: 0.5845
Epoch [8/25] Loss: 0.4305, Accuracy: 0.8712
Validation Accuracy: 0.5676
Epoch [9/25] Loss: 0.4347, Accuracy: 0.8716
Validation Accuracy: 0.5811
Epoch [10/25] Loss: 0.3956, Accuracy: 0.8837
Validation Accuracy: 0.5946
Epoch [11/25] Loss: 0.3512, Accuracy: 0.8947
Validation Accuracy: 0.5743
Epoch [12/25] Loss: 0.3113, Accuracy: 0.9045
Validation Accuracy: 0.6081
Epoch [13/25] Loss: 0.3021, Accuracy: 0.9000
Validation Accuracy: 0.5743
Epoch [14/25] Loss: 0.3266, Accuracy: 0.8992
Validation Accu

#PART2 FINAL RESULT

In [ ]:

from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import matplotlib.pyplot as plt

# Define the model
def create_model():
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 43)  # Adjust the final layer to the number of classes
    return model.to(device)

train_labels = [label for _, label in train_samples]
# Assuming train_labels contains all the labels from training dataset
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Mixup augmentation
def mixup_data(x, y, alpha=0.2):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model_wts = model.state_dict()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model_wts = model.state_dict()
            self.counter = 0

    def get_best_model_wts(self):
        return self.best_model_wts

# Training function
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=25, patience=15):
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    early_stopping = EarlyStopping(patience=patience, min_delta=0.01)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Apply Mixup
            inputs, targets_a, targets_b, lam = mixup_data(inputs, labels, alpha= 0.2)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (lam * predicted.eq(targets_a).sum().item() + (1 - lam) * predicted.eq(targets_b).sum().item())

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        model.eval()
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = val_correct / val_total
        val_losses.append(val_epoch_loss)
        val_accuracies.append(val_epoch_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, '
              f'Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}')

        early_stopping(val_epoch_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            model.load_state_dict(early_stopping.get_best_model_wts())
            break

        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_model_wts = model.state_dict()

        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(val_epoch_loss)  # Pass validation loss to scheduler
        else:
            scheduler.step()

    model.load_state_dict(best_model_wts)
    return model, train_losses, val_losses, train_accuracies, val_accuracies

# Hyperparameter configurations
learning_rates = [0.0001, 0.0003]
weight_decays = [1e-2, 1e-4]
schedulers = [
    lambda opt: ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=3, verbose=True),
    lambda opt: CosineAnnealingLR(opt, T_max=10, eta_min=1e-6)
]

# Store the results
results = []

for lr in learning_rates:
    for wd in weight_decays:
        for scheduler_fn in schedulers:
            print(f"Training with lr={lr}, weight_decay={wd}")
            model = create_model()
            optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
            scheduler = scheduler_fn(optimizer)

            model, train_losses, val_losses, train_accuracies, val_accuracies = train_model(
                model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=25, patience=15
            )

            results.append({
                'lr': lr,
                'weight_decay': wd,
                'scheduler': scheduler.__class__.__name__,
                'val_acc': max(val_accuracies),
                'train_losses': train_losses,
                'val_losses': val_losses,
                'train_accuracies': train_accuracies,
                'val_accuracies': val_accuracies
            })

# Display the results
import ace_tools as tools; tools.display_dataframe_to_user("Model Training Results", pd.DataFrame(results))


Training with lr=0.0001, weight_decay=0.01
Epoch [1/25] Train Loss: 2.3917, Train Acc: 0.3796, Val Loss: 1.3890, Val Acc: 0.6284
Epoch [2/25] Train Loss: 1.1664, Train Acc: 0.7305, Val Loss: 1.0612, Val Acc: 0.7331
Epoch [3/25] Train Loss: 0.8913, Train Acc: 0.8102, Val Loss: 1.1420, Val Acc: 0.6959
Epoch [4/25] Train Loss: 0.8082, Train Acc: 0.8482, Val Loss: 1.0301, Val Acc: 0.7500
Epoch [5/25] Train Loss: 0.7778, Train Acc: 0.8481, Val Loss: 0.9606, Val Acc: 0.7804
Epoch [6/25] Train Loss: 0.7179, Train Acc: 0.8653, Val Loss: 0.9126, Val Acc: 0.7804
Epoch [7/25] Train Loss: 0.6542, Train Acc: 0.8881, Val Loss: 0.9475, Val Acc: 0.7669
Epoch [8/25] Train Loss: 0.5650, Train Acc: 0.9072, Val Loss: 0.9913, Val Acc: 0.7601
Epoch [9/25] Train Loss: 0.6719, Train Acc: 0.8848, Val Loss: 1.1161, Val Acc: 0.7432
Epoch [10/25] Train Loss: 0.6335, Train Acc: 0.8952, Val Loss: 1.0516, Val Acc: 0.7770
Epoch 00010: reducing learning rate of group 0 to 1.0000e-05.
Epoch [11/25] Train Loss: 0.6001, 

ModuleNotFoundError: No module named 'ace_tools'